In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, confusion_matrix, classification_report
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler

In [2]:
# Tải data đã lưu
split_file = "splitted_data.pkl"
print("Tải dữ liệu đã lưu từ:", split_file)

with open(split_file, "rb") as f:
    data_splits = pickle.load(f)

# Lấy các tập dữ liệu từ dictionary
X_train = data_splits["X_train"]
X_val = data_splits["X_val"]
X_test = data_splits["X_test"]
y_train = data_splits["y_train"]
y_val = data_splits["y_val"]
y_test = data_splits["y_test"]
X_train_val = data_splits["X_train_val"]
y_train_val = data_splits["y_train_val"]

Tải dữ liệu đã lưu từ: splitted_data.pkl


In [3]:
# In thông tin về dữ liệu
print(f"\nKích thước tập huấn luyện (X_train): {X_train.shape}")
print(f"Kích thước tập validation (X_val): {X_val.shape}")
print(f"Kích thước tập kiểm tra (X_test): {X_test.shape}")
print(f"Kích thước tập huấn luyện + validation (X_train_val): {X_train_val.shape}")


Kích thước tập huấn luyện (X_train): (20238, 10)
Kích thước tập validation (X_val): (6746, 10)
Kích thước tập kiểm tra (X_test): (6747, 10)
Kích thước tập huấn luyện + validation (X_train_val): (26984, 10)


In [4]:
# Định nghĩa cross-validation phù hợp đảm bảo tỷ lệ fraud/non-fraud được giữ nguyên trong mỗi fold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [19]:
# 1. Tạo pipeline với TabNetClassifier
pipeline_tabnet = Pipeline([
    ('scaler', StandardScaler()),
    ('tabnet', TabNetClassifier(
        optimizer_fn=torch.optim.Adam,
        scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
        scheduler_params={
            'mode':      'max',      
            'factor':    0.5,
            'patience':  5,
            'min_lr':    1e-5,
            'verbose':   False      
        },
        verbose=0,
        seed=42
    ))
])


# 2. Định nghĩa Grid các siêu tham số cho TabNet
# Giảm số lượng tổ hợp để tìm kiếm hiệu quả hơn
tabnet_param_grid = {
    'tabnet__n_d':             [16, 32, 64],
    'tabnet__n_a':             [16, 32, 64],
    'tabnet__n_steps':         [3, 5],
    'tabnet__gamma':           [1.0, 1.5],
    'tabnet__lambda_sparse':   [1e-5, 1e-4],
    'tabnet__mask_type':       ['sparsemax'],
     'tabnet__optimizer_params': [
        {'lr': 1e-3},
        {'lr': 5e-3},
    ]
}

# 3. Sử dụng RandomizedSearchCV thay vì GridSearchCV để tiết kiệm thời gian
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    estimator=pipeline_tabnet,
    param_distributions=tabnet_param_grid,
    n_iter=20,              # Chỉ thử 20 tổ hợp siêu tham số
    cv=cv,
    scoring='average_precision',
    n_jobs=2,
    verbose=1,
    random_state=42
)

# 4. Chạy tìm siêu tham số
random_search.fit(
    X_train.values, y_train.values,
    tabnet__eval_set=[(X_val.values, y_val.values)],
    tabnet__max_epochs=200,
    tabnet__patience=15,
    tabnet__batch_size=512,
    tabnet__virtual_batch_size=128
)

# 5. Lấy kết quả tốt nhất
best_model_tabnet = random_search.best_estimator_
print("Best Parameters for TabNet:\n", random_search.best_params_)
print("Best Score:", random_search.best_score_)

# 6. Đánh giá trên tập Test
y_test_pred = best_model_tabnet.predict(X_test)

print("\nClassification Report (Test):")
print(classification_report(y_test, y_test_pred, zero_division=0))


Fitting 5 folds for each of 20 candidates, totalling 100 fits


C:\Users\ASUS\PycharmProjects\Graph Neural Network\.venv\lib\site-packages\sklearn\model_selection\_search.py:1107: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
  warnings.warn(



Early stopping occurred at epoch 19 with best_epoch = 4 and best_val_0_auc = 0.6149


C:\Users\ASUS\PycharmProjects\Graph Neural Network\.venv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Best Parameters for TabNet:
 {'tabnet__optimizer_params': {'lr': 0.005}, 'tabnet__n_steps': 3, 'tabnet__n_d': 64, 'tabnet__n_a': 16, 'tabnet__mask_type': 'sparsemax', 'tabnet__lambda_sparse': 0.0001, 'tabnet__gamma': 1.5}
Best Score: nan


C:\Users\ASUS\PycharmProjects\Graph Neural Network\.venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(



Classification Report (Test):
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6705
           1       0.09      0.05      0.06        42

    accuracy                           0.99      6747
   macro avg       0.54      0.52      0.53      6747
weighted avg       0.99      0.99      0.99      6747



In [11]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
# Tạo pipeline với TabNetClassifier
pipeline_tabnet = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('tabnet', TabNetClassifier(
        optimizer_fn=torch.optim.Adam,
        scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
        scheduler_params={
            'mode':      'max',      
            'factor':    0.5,
            'patience':  5,
            'min_lr':    1e-5,
            'verbose':   False      
        },
        verbose=0,
        seed=42
    ))
])


# 2. Định nghĩa Grid các siêu tham số cho TabNet
# Giảm số lượng tổ hợp để tìm kiếm hiệu quả hơn
tabnet_param_grid = {
    'tabnet__n_d':             [16, 32, 64],
    'tabnet__n_a':             [16, 32, 64],
    'tabnet__n_steps':         [3, 5],
    'tabnet__gamma':           [1.0, 1.5],
    'tabnet__lambda_sparse':   [1e-5, 1e-4],
    'tabnet__mask_type':       ['sparsemax'],
     'tabnet__optimizer_params': [
        {'lr': 1e-3},
        {'lr': 5e-3},
    ]
}

# 3. Sử dụng RandomizedSearchCV thay vì GridSearchCV để tiết kiệm thời gian
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    estimator=pipeline_tabnet,
    param_distributions=tabnet_param_grid,
    n_iter=20,              # Chỉ thử 20 tổ hợp siêu tham số
    cv=cv,
    scoring='average_precision',
    n_jobs=2,
    verbose=1,
    random_state=42
)

# 4. Chạy tìm siêu tham số
random_search.fit(
    X_train.values, y_train.values,
    tabnet__eval_set=[(X_val.values, y_val.values)],
    tabnet__max_epochs=200,
    tabnet__patience=20,
    tabnet__batch_size=256,
    tabnet__virtual_batch_size=64
)

# 5. Lấy kết quả tốt nhất
best_model_tabnet = random_search.best_estimator_
print("Best Parameters for TabNet - SMOTE:\n", random_search.best_params_)
print("Best Score:", random_search.best_score_)

# 6. Đánh giá trên tập Test
y_test_pred = best_model_tabnet.predict(X_test)

print("\nClassification Report (Test):")
print(classification_report(y_test, y_test_pred, zero_division=0))


Fitting 5 folds for each of 20 candidates, totalling 100 fits


KeyboardInterrupt: 

In [ ]:
# Sau khi tìm được siêu tham số tốt nhất
print("Retraining model with best parameters on full training data...")

# Tạo mô hình mới với siêu tham số tốt nhất
best_params = random_search.best_params_
final_model = TabNetClassifier(
    n_d=best_params['tabnet__n_d'],
    n_a=best_params['tabnet__n_a'],
    n_steps=best_params['tabnet__n_steps'],
    gamma=best_params['tabnet__gamma'],
    lambda_sparse=best_params['tabnet__lambda_sparse'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=best_params['tabnet__optimizer_params'],
    mask_type=best_params['tabnet__mask_type'],
    scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
    scheduler_params={'mode': 'max', 'patience': 5, 'min_lr': 1e-5, 'factor': 0.5},
    verbose=1,
    seed=42
)

# Huấn luyện trên toàn bộ tập train_val
X_train_val_array = X_train_val.values if hasattr(X_train_val, "values") else X_train_val
y_train_val_array = y_train_val.values if hasattr(y_train_val, "values") else y_train_val

final_model.fit(
    X_train=X_train_val_array, 
    y_train=y_train_val_array,
    eval_set=[(X_train_val_array, y_train_val_array)]  # Có thể thêm validation set nếu cần
)

# Đánh giá trên tập test với mô hình đã huấn luyện lại
y_test_pred = final_model.predict(X_test)
y_test_proba = final_model.predict_proba(X_test)[:, 1]

print("\nFinal Model Classification Report (Test):")
print(classification_report(y_test, y_test_pred, zero_division=0))
print("Final Test ROC AUC:           {:.4f}".format(roc_auc_score(y_test, y_test_proba)))
print("Final Test Average Precision: {:.4f}".format(average_precision_score(y_test, y_test_proba)))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. Lấy feature importances từ final_model đã train trên X_train_val
importances = final_model.feature_importances_

# 2. Xác định tên các đặc trưng từ X_train_val
if hasattr(X_train_val, 'columns'):
    feature_names = X_train_val.columns.tolist()
else:
    feature_names = [f"feature_{i}" for i in range(X_train_val.shape[1])]

# 3. Tạo DataFrame và sort
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})
importance_df = importance_df.sort_values('Importance', ascending=False)

# 4. Chọn top N
top_n = 20
top_df = importance_df.head(top_n)

# 5. Vẽ biểu đồ horizontal bar chart với matplotlib
plt.figure(figsize=(10, 6))
plt.barh(top_df['Feature'][::-1], top_df['Importance'][::-1])
plt.xlabel('Importance')
plt.title(f'Top {top_n} Feature Importances (trained on X_train_val)')
plt.tight_layout()
plt.show()
